# AI Financial Agent
https://www.youtube.com/watch?v=C8wvHKPlTls
Welcome to our tutorial on creating a financial agent application using LangGraph! This video is perfect for beginners eager to explore the power of AI in finance. 🌐📊 We'll guide you through creating tools, assigning them to an agent, and coordinating their work through a graph. Plus, we're adding a user interface with Gradio for an interactive experience. 🖥️✨

Discover how to check stock prices, get the latest news, detailed financial reports, and historical company data, all in one app. 📈🗞️ Whether you're new to LangGraph or looking to enhance your financial analysis skills, this video has everything you need.


## Prerequisites
Environment:
```
conda create -n langgraph python=3.11
conda activate langgraph
export OPENAI_API_KEY=
export POLYGON_API_KEY=
```

Servicios y API keys
- https://polygon.io/: Stock Market API Tools.
- https://platform.openai.com/ 



In [1]:
!conda create -n langgraph python=3.11

^C


In [2]:
!conda activate langgraph

In [5]:
# export OPENAI_API_KEY=
# export POLYGON_API_KEY=
import os
os.environ['OPENAI_API_KEY'] = 
os.environ['POLYGON_API_KEY'] = 


In [2]:
!pip install langgraph langchain langchain_community langchain_openai langchainhub polygon-api-client

  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
    --------------------------------------- 0.0/2.2 MB 640.0 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.2 MB 1.7 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.2 MB 2.0 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/2.2 MB 2.7 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.2 MB 3.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.8/2.2 MB 3.1 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.2 MB 3.3 MB/s eta 0:00:01
   --------------------- ------------------ 1.1/2.2 MB 3.3 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.2 MB 3.3 MB/s eta 0:00:01
   --------------------------- ------------ 1.5/2.2 MB 3.4 MB/s eta 0:00:01
   -----------------------

## Tool Creation

In [39]:
"""1. Define the tools our agent can use"""

import os  # noqa: E402
from langchain import hub  
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools import PolygonLastQuote, PolygonTickerNews, PolygonFinancials, PolygonAggregates

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4-0125-preview")
polygon = PolygonAPIWrapper()
tools= [
  PolygonLastQuote(api_wrapper=polygon),
  PolygonTickerNews(api_wrapper=polygon),
  PolygonFinancials(api_wrapper=polygon),
  PolygonAggregates(api_wrapper=polygon),
]
print(prompt.input_variables)

['agent_scratchpad', 'input']


## Agent Creation

In [28]:
"""2. Define the agent and helper functions"""
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

# Define the agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)


# Define the function to execute tools
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    observation = tool_to_use.invoke(agent_action.tool_input)
    # Añadimos el resultado de la accion al resultado
    data['intermediate_steps'].append((agent_action, observation))
    return data


## Graph Creation

In [29]:
"""3. Define the LangGraph"""

from langgraph.graph import END, Graph

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
  if isinstance(data["agent_outcome"], AgentFinish):
    return "exit"
  else:
    "continue"
    
    
workflow = Graph()
workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools) ## execute_tools(data)
workflow.set_entry_point("agent")
    
# De Agent --> Tools condicional
workflow.add_conditional_edges(
  "agent",
  should_continue,
  {
    "continue": "tools",
    "exit": END
  }
)

# De Tools --> Agent
workflow.add_edge("tools", "agent")
chain = workflow.compile()


In [46]:
result = chain.invoke({'input': "Cual ha sido el precio de BABA entre el 13/12/2023 al 14/06/2024?. Haz un analisis de la tendencia. Toma una decision (Vender, Comprar, Hold) para el dia 15/06/2024", "intermediate_steps": []})
output = result["agent_outcome"].return_values["output"]

print(output)

ValueError: API Error: {'ticker': 'BABA', 'queryCount': 126, 'resultsCount': 126, 'adjusted': True, 'results': [{'v': 21713154.0, 'vw': 70.69, 'o': 70.91, 'c': 71.46, 'h': 71.48, 'l': 70.12, 't': 1702443600000, 'n': 187300}, {'v': 25199289.0, 'vw': 72.5592, 'o': 71.47, 'c': 72.51, 'h': 73.24, 'l': 71.42, 't': 1702530000000, 'n': 219968}, {'v': 30787619.0, 'vw': 74.648, 'o': 74, 'c': 74.51, 'h': 75.545, 'l': 73.32, 't': 1702616400000, 'n': 258245}, {'v': 17233510.0, 'vw': 73.8358, 'o': 74.05, 'c': 73.84, 'h': 74.23, 'l': 73.21, 't': 1702875600000, 'n': 160369}, {'v': 20939297.0, 'vw': 75.2852, 'o': 74.4, 'c': 75.39, 'h': 76.01, 'l': 74.22, 't': 1702962000000, 'n': 180491}, {'v': 27540228.0, 'vw': 74.2659, 'o': 74.58, 'c': 73.35, 'h': 75.295, 'l': 73.33, 't': 1703048400000, 'n': 192505}, {'v': 20812090.0, 'vw': 75.4251, 'o': 74.86, 'c': 76.26, 'h': 76.29, 'l': 74.35, 't': 1703134800000, 'n': 166831}, {'v': 20004348.0, 'vw': 75.1845, 'o': 74.5, 'c': 75.28, 'h': 75.68, 'l': 74.44, 't': 170

In [36]:
!pip install gradio

  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached jsonschema-4.22.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/165.5 kB ? eta -:--:--
     ------------------- ------------------- 81.9/165.5 kB 1.5 MB/s eta 0:00:01
     -------------------------------------  163.8/165.5 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 165.5/165.5 kB 1.7 MB/s eta 0:00:00
  Using cached pyparsing-3.1.2-py3-none-any.whl.me

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
polygon-api-client 1.13.7 requires urllib3<2.0.0,>=1.26.9, but you have urllib3 2.2.1 which is incompatible.


In [38]:
"""1. Define the tools our agent can use"""
import os
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.tools import PolygonLastQuote, PolygonTickerNews, PolygonFinancials, PolygonAggregates
import gradio as gr

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4-0125-preview")

polygon = PolygonAPIWrapper()
tools = [
    PolygonLastQuote(api_wrapper=polygon),
    PolygonTickerNews(api_wrapper=polygon),
    PolygonFinancials(api_wrapper=polygon),
    PolygonAggregates(api_wrapper=polygon),
]

"""2. Define agent and helper functions"""
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

# Define the agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    observation = tool_to_use.invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data

"""3. Define the LangGraph"""
from langgraph.graph import END, Graph

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"

workflow = Graph()
workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "exit": END
    }
)
workflow.add_edge('tools', 'agent')
chain = workflow.compile()

def financial_agent(input_text):
    result = chain.invoke({"input": input_text, "intermediate_steps": []})
    output = result['agent_outcome'].return_values["output"]
    return output

iface = gr.Interface(
    fn=financial_agent,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=gr.Markdown(),
    title="Financial Agent",
    description="Financial Data Explorer: Leveraging Advanced API Tools for Market Insights"
)

iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
